In [ ]:
import os 
import cv2
import mediapipe as mp
import numpy as np 

In [ ]:

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False, min_detection_confidence=0.9, min_tracking_confidence=0.9
)

In [9]:
DATA_DIR = "./data"
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

num_classes = 27
dataset_size = 1000


In [15]:
# media pipe hands object
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False, min_detection_confidence=0.9, min_tracking_confidence=0.9
)

# function to create image with landmarks
def get_hand_landmarks_image(frame):
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # draw landmarks on blank 224x224 white image
    output_img = np.ones((224, 224, 3), np.uint8) * 255

    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                output_img,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(20,20,20), thickness=2, circle_radius=2),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(20,20,20), thickness=2, circle_radius=2),
            )
    return bool(results.multi_hand_landmarks), output_img

# function to get processed image with edge detection of hand by applying grayscale, blur and edge detection
def get_processed_image(frame):
    # minValue = 70
    # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # blur = cv2.GaussianBlur(gray,(5,5),2)

    # th3 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
    # ret, res = cv2.threshold(th3, minValue, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    res = cv2.cvtColor(res, cv2.COLOR_GRAY2BGR)

    return True,res

In [16]:
# data dir paths
unprocessed_data_dir = 'new/unprocessed'
processed_data_dir = 'new/new_unprocessed'
test_data_dir = 'new/test_unprocessed_new'

In [13]:
# function for preprocessing images
img_processing_func = get_processed_image

# loop through all directores in unprocessed_data_dir
for dir in os.listdir(unprocessed_data_dir):
    print(f'Processing {dir}...', end='')

    # create output dir and test dir
    output_dir = os.path.join(processed_data_dir, dir)
    os.makedirs(output_dir, exist_ok=True)

    # loop through all files in dir
    for file in os.listdir(os.path.join(unprocessed_data_dir, dir)):
        # load image
        image_rgb = cv2.imread(os.path.join(unprocessed_data_dir, dir, file))

        # process image
        ret, processed_image = img_processing_func(image_rgb)

        # save image
        if ret:
            cv2.imwrite(os.path.join(output_dir, file), processed_image)
        else:
            print(f'  Error in pre-processing {dir}/{file}')
    
    print(f'Done!')

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'new/unprocessed'

In [17]:
cap = cv2.VideoCapture(0)
# create dataset for ASL characters A-Z and blank

class_labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'blank']
n_images_per_class = 100

for index, label in enumerate(class_labels):
    # os.makedirs(os.path.join(processed_data_dir, str(index)), exist_ok=True)
    curr_img = 0
    print(f"Capturing images for {label}...", end='')
    while curr_img < n_images_per_class:
        ret, frame = cap.read()
        if not ret: continue

        # Preprocess the frame to match the input size of the model and scale the pixel values
        frame = frame[:, 80:560]
        frame = cv2.resize(frame, (224, 224))

        # get hand landmarks image with mediapipe
        success, hand_landmarks_img = img_processing_func(frame)

        # save image
        if success:
            cv2.imwrite(os.path.join(processed_data_dir, str(index), f'{curr_img+n_images_per_class+1}.jpg'), hand_landmarks_img)
            curr_img += 1

        preview_img = hand_landmarks_img.copy()
        frame_label = f'Finished {label}. press space' if curr_img == n_images_per_class else f'{label} {curr_img}/{n_images_per_class-1}' 
        cv2.putText(preview_img, frame_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 200), 2, cv2.LINE_AA)
        cv2.imshow('Preview', preview_img)

        if curr_img == n_images_per_class:
            print("Done!")
            while True:
                if cv2.waitKey(1) & 0xFF == ord(' '):
                    break
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        cv2.waitKey(200)

cap.release()
cv2.destroyAllWindows()

Capturing images for A...

TypeError: cannot unpack non-iterable bool object

In [18]:
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
# DEPENDENCIES ------>
import os
import cv2


DATA_DIR = "./data"
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

num_classes = 16
dataset_size = 200

cap = cv2.VideoCapture(0)
for i in range(num_classes):
    if not os.path.exists(os.path.join(DATA_DIR, str(i))):
        os.makedirs(os.path.join(DATA_DIR, str(i+10)))
    print("Collecting data")
    done = False

    while True:
        ret, frame = cap.read()
        cv2.putText(frame, "Press '1' ", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (255, 255, 255), 3, cv2.LINE_AA)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(25) == ord("1"):
            break

    count = 0
    while count < dataset_size:
        ret, frame = cap.read()
        cv2.imshow("Frame", frame)
        cv2.waitKey(25)
        cv2.imwrite(os.path.join(DATA_DIR, str(i+10), "{}.jpg".format(count)), frame)
        count += 1
        print("Images saved: ", count)

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 